In [ ]:
letters = ["a", "b", "c", "d", "e", "f", "g", "h"]
numbers = list(range(1, 10)) # [1..9]
MOVE_DICTIONARY = {}
cumulative = 0
for i in range(8):
    for j in range(8):
        for k in range(8):
            for w in range (8):
                if (i == k and j == w):
                    cumulative += 1
                    continue
                from_square = f"{letters[i]}{numbers[j]}"
                to_square = f"{letters[k]}{numbers[w]}"
                MOVE_DICTIONARY[f"{from_square}{to_square}"] = (i * 8**3) + (j * 8**2) + (k * 8) + w - cumulative
REVERSE_MOVE_DICTIONARY = {
    value: key for key,value in MOVE_DICTIONARY.items()
}

In [ ]:
import csv
from io import StringIO
import pandas as pd
import numpy as np
import ast
# torch.save(model.state_dict(), "pieces_model_skip_initial_full_dataset.pth")
test = pd.read_csv(
    '../dataset/processed/results_skip_initial.csv', 
    nrows = 20, 
    names = HEADERS,
)
entry = test.iloc[0]
bitmaps = np.array(ast.literal_eval(entry["bitmaps"]))
side_channel = np.ones((1, 8, 8)) if entry["sideToPlay"] == "True" else np.zeros((1, 8, 8))

bitmaps = np.concatenate((bitmaps, side_channel), axis=0)
X = bitmaps
played_move = entry["movePlayed"]

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = CompleteChessBotNetwork().to(device)
trained_model.load_state_dict(torch.load("./CompleteModel_SKIP_INITIAL_epoch_76.pth"))
trained_model.eval()  # Set model to evaluation mode

def predict_move(bitmaps):
    bitmaps  = np.expand_dims(bitmaps, axis=0)
    X_tensor = torch.tensor(bitmaps, dtype=torch.float32).to(device)
    
    with torch.no_grad():
        logits = trained_model(X_tensor)
    
    logits = logits.squeeze(0)  # Remove batch dimension
    
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()  # Convert to probabilities
    
    return probabilities

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Mapping of piece letters to unicode chess symbols
PIECE_UNICODE = {
    'P': '♙', 'N': '♘', 'B': '♗', 'R': '♖', 'Q': '♕', 'K': '♔',
    'p': '♟︎', 'n': '♞', 'b': '♝', 'r': '♜', 'q': '♛', 'k': '♚'
}

def plot_board(bitmaps, move_probs=None):
    board = np.zeros((8, 8, 3))
    colors = [(240/255, 217/255, 181/255), (181/255, 136/255, 99/255)]

    fig, ax = plt.subplots(figsize=(8, 8))

    # Draw squares
    for i in range(8):
        for j in range(8):
            color = colors[(i + j) % 2]
            rect = plt.Rectangle((j, 7-i), 1, 1, facecolor=color)
            ax.add_patch(rect)

    # Draw pieces
    piece_symbols = [ 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
    for idx, symbol in enumerate(piece_symbols):
        channel = bitmaps[idx]
        for i in range(8):
            for j in range(8):
                if channel[i, j] == 1:
                    piece_unicode = PIECE_UNICODE[symbol]
                    ax.text(j + 0.5, 7 - i + 0.5, piece_unicode, fontsize=48, ha='center', va='center')

    # Draw move probabilities
    if move_probs is not None:
        for i in range(8):
            for j in range(8):
                prob = move_probs[i, j]
                if prob > 0:  # Show only significant probs
                    ax.text(j + 0.5, 7 - i + 0.1, f"{prob:.2f}", fontsize=8, color="red", ha='center')
                    # ax.text(j + 0.5, i + 0.8, f"{i*8 + j}", fontsize=8, color="red", ha='center')

    # Add a-h and 1-8 labels
    # files = ['h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']
    files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    ranks = ['8', '7', '6', '5', '4', '3', '2', '1']
    for i in range(8):
        # Bottom labels
        ax.text(i + 0.5, -0.3, files[i], ha='center', va='center', fontsize=14)
        # Left side labels
        ax.text(-0.3, i + 0.5, ranks[7-i], ha='center', va='center', fontsize=14)

    ax.set_xlim(0, 8)
    ax.set_ylim(0, 8)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()


In [ ]:
probabilities = predict_move(bitmaps)  # model output
print(probabilities)

# Recebmos as classes dos moves: converter para moves e depois ordernar os moves por probabilidade
moves = [REVERSE_MOVE_DICTIONARY[idx] for idx, prob in enumerate(probabilities)]
best_moves = sorted(
    zip(moves, probabilities),
    key=lambda x: x[1], # Sort by probability
    reverse=True
)
print(best_moves[:10])
plot_board(bitmaps)

In [ ]:
from IPython.display import clear_output, display
import time

board = chess.Board()
probs = np.zeros((8, 8))
input_tensor = board_to_tensor(board)
plot_board(input_tensor)

while not board.is_game_over():


    user_move = input("Your move (e.g., e2e4): ")
    if user_move == "exit":
        break
    try:
        board.push_san(user_move)
    except Exception as e:
        print("Invalid move. Try again.")
        continue
    clear_output(wait=True)

    # Convert board to bitmap
    bitmaps = board_to_tensor(board)
    side_channel = np.ones((1, 8, 8)) if board.turn == chess.WHITE else np.zeros((1, 8, 8))
    input_tensor = np.concatenate((bitmaps, side_channel), axis=0)

    # Model predicts best move
    probs = predict_move(input_tensor)
    best_square_idx = np.argmax(probs, keepdims=True)
    # Recebmos as classes dos moves: converter para moves e depois ordernar os moves por probabilidade
    moves = [REVERSE_MOVE_DICTIONARY[idx] for idx, prob in enumerate(probabilities)]
    best_moves = sorted(
        zip(moves, probabilities),
        key=lambda x: x[1], # Sort by probability
        reverse=True
    )

    for move in best_moves:
        try:
            board.push_uci(move[0])
            break
        except:
            continue

    # Convert board to bitmap
    bitmaps = board_to_tensor(board)
    side_channel = np.ones((1, 8, 8)) if board.turn == chess.WHITE else np.zeros((1, 8, 8))
    input_tensor = np.concatenate((bitmaps, side_channel), axis=0)
    plot_board(input_tensor)